In [1]:
# Core and data handling
import numpy as np
import pandas as pd
import os
from statistics import stdev

# Visualization
import matplotlib.pyplot as plt
%matplotlib inline

# Machine Learning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.utils import compute_class_weight
from sklearn.metrics import (
    confusion_matrix, mean_squared_error, precision_recall_curve,
    precision_score, recall_score, accuracy_score, f1_score,
    roc_auc_score, average_precision_score
)

# Imbalanced learning
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SVMSMOTE
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
import lightgbm as lgb

# XGBoost
import xgboost as xgb

# Deep Learning
import torch
import torchvision
import torchvision.utils
import tensorflow as tf

# Dataset
from sklearn.datasets import make_classification

# SVM
from sklearn.svm import SVC
## Functions defined by me 
from model_utils import *

# Check GPU availability
print(f"PyTorch CUDA available: {torch.cuda.is_available()}")

# If you want to drop duplicates in a pandas DataFrame, you would do:
# df = df.drop_duplicates()

ModuleNotFoundError: No module named 'model_utils'

In [ ]:
df=pd.read_csv('creditcard.csv')

In [ ]:
df=df.iloc[:,1:]

## 1. Read Data 

In [ ]:
df.iloc[:,-1:]['Class']

In [6]:
df = pd.read_csv('creditcard.csv')

# Take a 10% stratified sample
stratified_sample = df.groupby('Class', group_keys=False).apply(
    lambda x: x.sample(frac=0.2, random_state=11)
)


stratified_sample=stratified_sample.iloc[:,1:]

X=stratified_sample.iloc[:,:-1]
y=stratified_sample.iloc[:,-1:]

y=y['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=11,stratify=y)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_22808\2641008765.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stratified_sample = df.groupby('Class', group_keys=False).apply(


In [7]:
y_train.sum()/y_train.shape[0]

np.float64(0.00173053772070626)

In [8]:
y_test.sum()/y_test.shape[0]

np.float64(0.0016969980689332319)

# 1. Modelling with LightGBM algorithm with just using SMOTE to oversample train data 
## For each configuration, model is trained for 20 times and the averages of the output scores are taken.

In [9]:
# Define an SVM with RBF kernel
#svm_estimator = SVC(kernel='rbf', gamma='scale', random_state=11)

# Apply SVM-SMOTE with the RBF kernel SVM
svm_smote = SMOTE(
    #svm_estimator=svm_estimator,  # Use the RBF kernel SVM
    random_state=11
)
X_train_resampled, y_train_resampled = svm_smote.fit_resample(X_train, y_train)

In [10]:
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    precision_recall_curve, accuracy_score, roc_auc_score, 
    average_precision_score, recall_score, f1_score, confusion_matrix
)
import pandas as pd
import numpy as np

def calculate_metrics(y_test, y_pred_proba):
    """Calculate binary classification metrics given true labels and predicted probabilities."""
    y_pred_binary = [1 if p > 0.5 else 0 for p in y_pred_proba]

    accuracy = accuracy_score(y_test, y_pred_binary)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    pr_auc = average_precision_score(y_test, y_pred_proba)
    recall = recall_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_binary).ravel()
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    fp_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
    g_mean = np.sqrt(recall * specificity)

    metrics_dict = {
        'accuracy': accuracy,
        'roc_auc': roc_auc,
        'pr_auc': pr_auc,
        'recall': recall,
        'f1': f1,
        'specificity': specificity,
        'fp_rate': fp_rate,
        'g_mean': g_mean
    }

    return metrics_dict

def callf1(X_train, y_train, X_test, y_test):
    results = {}

    ### ------------------------------
    ### LightGBM
    ### ------------------------------
    dtrain = lgb.Dataset(X_train, label=y_train)
    params = {
        "objective": "binary",
        "metric": "auc",
        "boosting_type": "gbdt",
        "device": "gpu"
    }
    model = lgb.train(params, dtrain)
    y_pred_proba = model.predict(X_test)
    results['LightGBM'] = calculate_metrics(y_test, y_pred_proba)

    ### ------------------------------
    ### XGBoost RF
    ### ------------------------------
    rf_model = xgb.XGBClassifier(
        tree_method="gpu_hist",
        predictor="gpu_predictor",
        objective="binary:logistic",
        booster="gbtree",
        num_parallel_tree=100,
        subsample=0.8,
        colsample_bytree=0.8,
        n_estimators=100
    )
    rf_model.fit(X_train, y_train)
    y_pred_proba_rf = rf_model.predict_proba(X_test)[:, 1]
    results['XGBoost_RF'] = calculate_metrics(y_test, y_pred_proba_rf)

    ### ------------------------------
    ### XGBoost GBDT
    ### ------------------------------
    xgb_model = xgb.XGBClassifier(
        tree_method="gpu_hist",
        predictor="gpu_predictor",
        objective="binary:logistic",
        booster="gbtree",
        n_estimators=100
    )
    xgb_model.fit(X_train, y_train)
    y_pred_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]
    results['XGBoost_GBDT'] = calculate_metrics(y_test, y_pred_proba_xgb)

    ### ------------------------------
    ### Logistic Regression
    ### ------------------------------
    lr = LogisticRegression(max_iter=1000)
    lr.fit(X_train, y_train)
    y_pred_proba_lr = lr.predict_proba(X_test)[:, 1]
    results['LogisticRegression'] = calculate_metrics(y_test, y_pred_proba_lr)

    ### ------------------------------
    ### Combine as DataFrame
    ### ------------------------------
    results_df = pd.DataFrame(results)
    results_df.insert(0, 'metric', results_df.index)
    results_df.reset_index(drop=True, inplace=True)

    return results_df


In [11]:

sum=0
for i in range(1):
    r=callf1(X_train_resampled,y_train_resampled.ravel(),X_test,y_test.ravel())


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_22808\411211134.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r=callf1(X_train_resampled,y_train_resampled.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 39803, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79606, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.006926 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:00:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:00:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:01:01] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

In [12]:
sum=0
for i in range(1):
    
    rson=callf1(X_train , y_train.ravel(),X_test,y_test.ravel())


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_22808\2770109251.py:4: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  rson=callf1(X_train , y_train.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 69, number of negative: 39803
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 39872, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (1.22 MB) transferred to GPU in 0.004632 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001731 -> initscore=-6.357591
[LightGBM] [Info] Start training from score -6.357591


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:01:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:01:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:01:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

In [13]:
rson

,metric,LightGBM,XGBoost_RF,XGBoost_GBDT,LogisticRegression
0,accuracy,0.994382,0.999824,0.999707,0.999473
1,roc_auc,0.609065,0.999962,0.999970,0.999454
2,pr_auc,0.116612,0.977162,0.981792,0.870430
3,recall,0.517241,0.931034,0.862069,0.827586
4,f1,0.238095,0.947368,0.909091,0.842105
5,specificity,0.995193,0.999941,0.999941,0.999766
6,fp_rate,0.004807,0.000059,0.000059,0.000234
7,g_mean,0.717464,0.964873,0.928449,0.909611


In [14]:
r

,metric,LightGBM,XGBoost_RF,XGBoost_GBDT,LogisticRegression
0,accuracy,0.999239,0.999473,0.999239,0.965650
1,roc_auc,0.999643,0.999836,0.999838,0.999721
2,pr_auc,0.881328,0.942439,0.939463,0.839278
3,recall,0.896552,0.931034,0.896552,1.000000
4,f1,0.800000,0.857143,0.800000,0.089922
5,specificity,0.999414,0.999590,0.999414,0.965592
6,fp_rate,0.000586,0.000410,0.000586,0.034408
7,g_mean,0.946587,0.964703,0.946587,0.982645


# 2. Modelling with LightGBM algorithm with  using SMOTE-GAN hybrids to oversample train data 


In [15]:
from gan_smoteenn import gan_augment_with_enn
from cgan_smoteenn import cgan_augment_with_enn
from wgan_smoteenn import wgan_gp_augment_with_enn

from smotified_gan_tomek import gan_augment_with_existing_resampling
from smotified_cgan_tomek import cgan_augment_with_existing_resampling
from smotified_wgan_tomek import wgan_augment_with_existing_resampling


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
Xn, yn = gan_augment_with_existing_resampling(X_train, y_train,X_train_resampled, y_train_resampled ,apply_tomek=False)
Xn_cgan, yn_cgan = cgan_augment_with_existing_resampling(X_train, y_train,X_train_resampled, y_train_resampled,apply_tomek=False)
Xn_wgan, yn_wgan = wgan_augment_with_existing_resampling(X_train, y_train,X_train_resampled, y_train_resampled,apply_tomek=False)

C:\Users\Onur Yaman\Tez\smotified_gan_tomek.py:85: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_tr = y_train.ravel()
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  8.67it/s]


Epoch 1, Step 1 | Gen Loss: 1.4665696024894714, Disc Loss: 1.5064101815223694


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  8.56it/s]


Epoch 2, Step 2 | Gen Loss: 0.7338427305221558, Disc Loss: 0.7447762489318848


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  8.46it/s]


...

Epoch 197 | Gen Loss: -1.9138, Critic Loss: -5.6501


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 18.16it/s]


Epoch 198 | Gen Loss: -2.1216, Critic Loss: -5.8636


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 20.44it/s]

Epoch 199 | Gen Loss: -0.0858, Critic Loss: -6.1632


In [ ]:
sum=0
for i in range(1):
    r2_notomek=callf1(Xn, yn.ravel(),X_test,y_test.ravel())
    r3_notomek=callf1(Xn_cgan, yn_cgan.ravel(),X_test,y_test.ravel())
    r4_notomek=callf1(Xn_wgan, yn_wgan.ravel(),X_test,y_test.ravel())
from imblearn.under_sampling import TomekLinks

tomek = TomekLinks()
Xn_tomek, yn_tomek = tomek.fit_resample(Xn,yn)
tomek = TomekLinks()
Xn_cgan_tomek, yn_cgan_tomek = tomek.fit_resample(Xn_cgan, yn_cgan)
tomek = TomekLinks()
Xn_wgan_tomek, yn_wgan_tomek = tomek.fit_resample(Xn_wgan, yn_wgan)

for i in range(1):
    r2_tomek=callf1(Xn_tomek, yn_tomek.ravel(),X_test,y_test.ravel())
    r3_tomek=callf1(Xn_cgan_tomek, yn_cgan_tomek.ravel(),X_test,y_test.ravel())
    r4_tomek=callf1(Xn_wgan_tomek, yn_wgan_tomek.ravel(),X_test,y_test.ravel())


In [23]:
Xn_2, yn_2 = gan_augment_with_enn(X_train, y_train ,apply_enn=True) #EN İYİ ENN İLE SONUÇ VERDİ
Xn_cgan_2, yn_cgan_2 = cgan_augment_with_enn(X_train, y_train,apply_enn=True)
Xn_wgan_2, yn_wgan_2 = wgan_gp_augment_with_enn(X_train, y_train,apply_enn=True)
sum=0
for i in range(1):
    r2=callf1(Xn_2, yn_2.ravel(),X_test,y_test.ravel())
    r3=callf1(Xn_cgan_2, yn_cgan_2.ravel(),X_test,y_test.ravel())
    r4=callf1(Xn_wgan_2, yn_wgan_2.ravel(),X_test,y_test.ravel())

C:\Users\Onur Yaman\Tez\gan_smoteenn.py:84: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_tr = y_train.ravel()
                                                                                                                       

Epoch 0 | Gen Loss: 0.6698, Disc Loss: 0.7167


Epoch 1 | Gen Loss: 0.6701, Disc Loss: 0.7120


Epoch 2 | Gen Loss: 0.6704, Disc Loss: 0.7074


Epoch 3 | Gen Loss: 0.6710, Disc Loss: 0.7028


...

C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:06:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:06:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:06:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

[LightGBM] [Info] Number of positive: 207, number of negative: 39766
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 39973, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (1.22 MB) transferred to GPU in 0.004542 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.005178 -> initscore=-5.258049
[LightGBM] [Info] Start training from score -5.258049
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fur

C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:06:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:06:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:07:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

[LightGBM] [Info] Number of positive: 207, number of negative: 39764
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 39971, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (1.22 MB) transferred to GPU in 0.004537 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.005179 -> initscore=-5.257998
[LightGBM] [Info] Start training from score -5.257998
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fur

C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:07:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:07:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [01:07:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

In [85]:
df_merged=pd.concat([rson[['metric','LightGBM']],r['LightGBM'],r2['LightGBM'],r3['LightGBM'],r4['LightGBM'],r2_tomek['LightGBM'],r3_tomek['LightGBM'],r4_tomek['LightGBM']],axis=1)

In [99]:
df_merged.columns=['Metric','No SMOTE','Basic SMOTE','GAN-ENN Hybrid','CGAN-ENN Hybrid','WGAN_GP-ENN Hybrid','SMOTE-GAN-TOMEK Hybrid','SMOTE-CGAN-TOMEK Hybrid','SMOTE-WGAN_GP-TOMEK Hybrid']

In [1]:
df_merged

NameError: name 'df_merged' is not defined

## 3.  Modelling with LightGBM algorithm with SMOTEENN with adaptive distance weight variants 

In [91]:
from adaptive_smoteenn import smoteenn_with_weighted_voting


In [92]:
from adaptive_smoteenn import smoteenn_with_weighted_voting
distance_algos=['gaussian','inverse','exponential','rank','adaptive']
results=[]
for algo in distance_algos:
    X_resampled, y_resampled = smoteenn_with_weighted_voting(
        X_train, y_train,
        strategy=str(algo),          # 'inverse', 'exponential', 'rank', 'adaptive'
        n_neighbors_smote=5,
        n_neighbors_enn=5,
        alpha=1.0,
        beta=1.0,
        sigma=1.0,                    # for gaussian weighting
        random_state=42
    )
    
    sum=0
    
    for i in range(1):
        
        r=callf1(X_resampled, y_resampled.ravel(),X_test,y_test.ravel())

    results.append(r)


Original class distribution: Counter({np.int64(0): 39803, np.int64(1): 69})
After SMOTE: Counter({np.int64(0): 39803, np.int64(1): 39803})
After distance-weighted ENN: Counter({np.int64(1): 39803, np.int64(0): 39746})


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_22808\690489541.py:20: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r=callf1(X_resampled, y_resampled.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Number of positive: 39803, number of negative: 39746
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79549, number of used features: 29
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.006596 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500358 -> initscore=0.001433
[LightGBM] [Info] Start training from score 0.001433


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:13:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:13:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:14:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

Original class distribution: Counter({np.int64(0): 39803, np.int64(1): 69})
After SMOTE: Counter({np.int64(0): 39803, np.int64(1): 39803})
After distance-weighted ENN: Counter({np.int64(1): 39803, np.int64(0): 39721})
[LightGBM] [Info] Number of positive: 39803, number of negative: 39721
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79524, number of used features: 29


...

C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:23:11] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:23:11] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:23:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

Original class distribution: Counter({np.int64(0): 39803, np.int64(1): 69})
After SMOTE: Counter({np.int64(0): 39803, np.int64(1): 39803})
After distance-weighted ENN: Counter({np.int64(1): 39803, np.int64(0): 39721})
[LightGBM] [Info] Number of positive: 39803, number of negative: 39721
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 79524, number of used features: 29


C:\Users\Onur Yaman\AppData\Local\Temp\ipykernel_22808\690489541.py:20: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  r=callf1(X_resampled, y_resampled.ravel(),X_test,y_test.ravel())


[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 2060 with Max-Q Design, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 29 dense feature groups (2.43 MB) transferred to GPU in 0.006456 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500516 -> initscore=0.002062
[LightGBM] [Info] Start training from score 0.002062


C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:26:22] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:26:22] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "predictor" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Onur Yaman\anaconda3\envs\onur_1\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:26:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xg

In [93]:
import pandas as pd

# Initialize a list to store processed results
processed_results = []

for i, algo in enumerate(distance_algos):
    # Convert the results to a Series with metric names as index
    result_series = results[i][['metric','LightGBM']].set_index('metric')['LightGBM']
    result_series.name = algo  # Name the series after the algorithm
    processed_results.append(result_series)

# Concatenate all series into a DataFrame
final_results = pd.concat(processed_results, axis=1)

# Display the final DataFrame


In [94]:
final_results.columns='SMOTE_ENN_'+final_results.columns

In [95]:
final_results.reset_index(drop=True,inplace=True)

In [96]:
final_results

,SMOTE_ENN_gaussian,SMOTE_ENN_inverse,SMOTE_ENN_exponential,SMOTE_ENN_rank,SMOTE_ENN_adaptive
0,0.999298,0.999181,0.999356,0.999415,0.999181
1,0.999662,0.999721,0.999751,0.999802,0.999723
2,0.862363,0.867605,0.867842,0.869428,0.868667
3,0.965517,0.965517,0.965517,0.965517,0.965517
4,0.823529,0.800000,0.835821,0.848485,0.800000
5,0.999355,0.999238,0.999414,0.999472,0.999238
6,0.000645,0.000762,0.000586,0.000528,0.000762
7,0.982291,0.982233,0.982319,0.982348,0.982233


In [97]:
df_all=pd.concat([df_merged,final_results],axis=1)

In [98]:
df_all

,Metric,No SMOTE,Basic SMOTE,SMOTE-GAN-ENN Hybrid,SMOTE-CGAN-ENN Hybrid,SMOTE-WGAN_GP-ENN Hybrid,GAN-TOMEK Hybrid,CGAN-TOMEK Hybrid,WGAN_GP-TOMEK Hybrid,SMOTE_ENN_gaussian,SMOTE_ENN_inverse,SMOTE_ENN_exponential,SMOTE_ENN_rank,SMOTE_ENN_adaptive
0,accuracy,0.994382,0.999239,0.999532,0.999532,0.999298,0.999766,0.999473,0.999122,0.999298,0.999181,0.999356,0.999415,0.999181
1,roc_auc,0.609065,0.999721,0.999865,0.999895,0.931095,0.999962,0.999869,0.999347,0.999662,0.999721,0.999751,0.999802,0.999723
2,pr_auc,0.116612,0.867605,0.869435,0.916352,0.813463,0.980231,0.923216,0.924247,0.862363,0.867605,0.867842,0.869428,0.868667
3,recall,0.517241,0.965517,0.931034,0.896552,0.896552,0.931034,0.758621,0.896552,0.965517,0.965517,0.965517,0.965517,0.965517
4,f1,0.238095,0.811594,0.870968,0.866667,0.812500,0.931034,0.830189,0.776119,0.823529,0.800000,0.835821,0.848485,0.800000
5,specificity,0.995193,0.999297,0.999648,0.999707,0.999472,0.999883,0.999883,0.999297,0.999355,0.999238,0.999414,0.999472,0.999238
6,fp_rate,0.004807,0.000703,0.000352,0.000293,0.000528,0.000117,0.000117,0.000703,0.000645,0.000762,0.000586,0.000528,0.000762
7,g_mean,0.717464,0.982262,0.964732,0.946725,0.946614,0.964845,0.870937,0.946531,0.982291,0.982233,0.982319,0.982348,0.982233
